## Sisestused

Eelmises näites töötasime kõrgedimensionaalsete sõnakottide vektoritega, mille pikkus oli `vocab_size`, ja teisendasime madaladimensionaalsed positsiooniesituse vektorid otseselt hõredaks ühekuumaks esitusviisiks. See ühekuum esitusviis ei ole mälusäästlik. Lisaks käsitletakse iga sõna üksteisest sõltumatult, mistõttu ühekuum kodeeritud vektorid ei väljenda sõnade semantilisi sarnasusi.

Selles osas jätkame **News AG** andmestiku uurimist. Alustuseks laadime andmed ja võtame mõned definitsioonid eelmisest osast.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Mis on embedding?

**Embedding** tähendab sõnade esitamist madalama dimensiooniga tihedate vektoritega, mis peegeldavad sõna semantilist tähendust. Hiljem arutame, kuidas luua tähenduslikke sõnaembeddingeid, kuid praegu mõtleme embedding'ist lihtsalt kui viisist vähendada sõnavektori dimensioonilisust.

Embedding-kiht võtab sisendiks sõna ja annab väljundiks vektori, mille suurus on määratud `embedding_size`. Mingis mõttes on see väga sarnane `Dense`-kihile, kuid selle asemel, et võtta sisendiks ühekuumkoodiga vektor, suudab see kasutada sõna numbrit.

Kasutades embedding-kihti meie võrgu esimeseks kihiks, saame liikuda sõnakottide mudelilt **embedding-koti** mudelile, kus esmalt teisendame iga sõna tekstis vastavaks embedding'iks ja seejärel arvutame nende embedding'ite üle mingi koondfunktsiooni, näiteks `sum`, `average` või `max`.

![Pilt, mis näitab embedding-klassiﬁkaatorit viie järjestikuse sõna jaoks.](../../../../../translated_images/et/embedding-classifier-example.b77f021a7ee67eee.webp)

Meie klassifitseeriva närvivõrgu kihid on järgmised:

* `TextVectorization` kiht, mis võtab sisendiks stringi ja annab väljundiks tokenite numbrite tensor. Määrame mõistliku sõnavara suuruse `vocab_size` ja ignoreerime harvemini kasutatavaid sõnu. Sisendi kuju on 1 ja väljundi kuju on $n$, kuna tulemuseks saame $n$ tokenit, millest igaüks sisaldab numbreid vahemikus 0 kuni `vocab_size`.
* `Embedding` kiht, mis võtab $n$ numbrit ja vähendab iga numbri tihedaks vektoriks kindla pikkusega (näiteks 100 meie näites). Seega muudetakse sisendi tensor kujuga $n$ tensoriks kujuga $n\times 100$.
* Koondamise kiht, mis arvutab selle tensori keskmise esimese telje järgi, st arvutab kõigi $n$ sisendtensori keskmise, mis vastavad erinevatele sõnadele. Selle kihi rakendamiseks kasutame `Lambda`-kihti ja anname sellele funktsiooni keskmise arvutamiseks. Väljundil on kuju 100 ja see on kogu sisendjärjestuse numbriline esitus.
* Lõplik `Dense` lineaarne klassifitseerija.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` väljatrükis, **output shape** veerus, tähistab esimene tensorimõõde `None` minibatch'i suurust ja teine mõõde tähistab tokenite järjestuse pikkust. Kõik minibatch'is olevad tokenite järjestused on erineva pikkusega. Järgmises osas arutame, kuidas sellega toime tulla.

Nüüd treenime võrku:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Märkus**: me loome vektoreerija andmete alamhulga põhjal. Seda tehakse protsessi kiirendamiseks, kuid see võib viia olukorrani, kus kõik tekstis olevad tokenid ei ole sõnavaras esindatud. Sellisel juhul need tokenid jäetakse tähelepanuta, mis võib põhjustada veidi madalamat täpsust. Kuid päriselus annab tekstialamhulk sageli hea sõnavara hinnangu.


### Töötamine muutuvate järjestuse suurustega

Vaatame, kuidas treenimine minibatchides toimub. Eelnevas näites on sisendtensoril dimensioon 1 ja me kasutame 128-pikkuseid minibatche, nii et tensori tegelik suurus on $128 \times 1$. Kuid iga lause tokenite arv on erinev. Kui rakendame `TextVectorization` kihti ühele sisendile, on tagastatud tokenite arv erinev, sõltuvalt sellest, kuidas tekst on tokeniseeritud:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Kuid kui rakendame vektoreerijat mitmele järjestusele, peab see tootma ristkülikukujulise tensori, seega täidab see kasutamata elemendid PAD-tokniga (mis meie puhul on null):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Siin näeme sisestusi:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Märkus**: Täitematerjali hulga vähendamiseks on mõnel juhul mõistlik järjestada kõik andmekogumi järjestused kasvava pikkuse järjekorras (täpsemalt, vastavalt tokenite arvule). See tagab, et iga minibatch sisaldab sarnase pikkusega järjestusi.


## Semantilised esitlused: Word2Vec

Eelmises näites õppis sisendkiht sõnu vektoriteks teisendama, kuid need vektorid ei kandnud semantilist tähendust. Oleks kasulik õppida vektorite esitusviis, kus sarnased sõnad või sünonüümid vastavad vektoritele, mis on üksteisele lähedal mingi vektori kauguse (näiteks eukleidiline kaugus) järgi.

Selleks peame oma sisendmudeli eelnevalt treenima suure tekstikogu peal, kasutades tehnikat nagu [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). See põhineb kahel peamisel arhitektuuril, mida kasutatakse sõnade hajutatud esituse loomiseks:

 - **Järjepidev sõnakott** (Continuous bag-of-words, CBoW), kus mudelit treenitakse ennustama sõna ümbritseva konteksti põhjal. Arvestades ngrammi $(W_{-2},W_{-1},W_0,W_1,W_2)$, on mudeli eesmärk ennustada $W_0$ $(W_{-2},W_{-1},W_1,W_2)$ põhjal.
 - **Järjepidev vahelejätu-gramm** (Continuous skip-gram) on CBoW vastand. Mudel kasutab ümbritsevat kontekstiakent, et ennustada praegust sõna.

CBoW on kiirem, samas kui vahelejätu-gramm on aeglasem, kuid esindab haruldasi sõnu paremini.

![Pilt, mis näitab nii CBoW kui ka Skip-Gram algoritme sõnade vektoriteks teisendamiseks.](../../../../../translated_images/et/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Et katsetada Word2Vec sisendit, mis on eelnevalt treenitud Google News andmestiku peal, saame kasutada **gensim** teeki. Allpool otsime sõnu, mis on kõige sarnasemad sõnale 'neural'.

> **Märkus:** Kui loote esimest korda sõnavektoreid, võib nende allalaadimine võtta aega!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Saame samuti sõnast vektori sisestuse välja võtta, et kasutada seda klassifitseerimismudeli treenimisel. Sisestusel on 300 komponenti, kuid siin näitame selguse huvides ainult vektori esimesi 20 komponenti:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Suurepärane asi semantiliste sisendite juures on see, et saate vektorkodeeringut manipuleerida semantika põhjal. Näiteks võime paluda leida sõna, mille vektorrepresentatsioon on võimalikult lähedal sõnadele *kuningas* ja *naine*, ning võimalikult kaugel sõnast *mees*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ülaltoodud näide kasutab mõningast sisemist GenSymi maagiat, kuid aluseks olev loogika on tegelikult üsna lihtne. Huvitav asi sisendvektorite juures on see, et saate sisendvektoritel teha tavalisi vektorioperatsioone, mis peegeldavad sõnade **tähenduste** operatsioone. Ülaltoodud näidet saab väljendada vektorioperatsioonide abil: arvutame vektori, mis vastab **KUNINGAS-MEES+NAINE** (operatsioonid `+` ja `-` tehakse vastavate sõnade vektorrepresentatsioonidel), ja seejärel leiame sõnastikust sellele vektorile kõige lähedasema sõna:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Me pidime lisama väikese koefitsiendi *mees* ja *naine* vektoritele - proovi need eemaldada, et näha, mis juhtub.

Kõige lähedasema vektori leidmiseks kasutame TensorFlow tööriistu, et arvutada kauguste vektor meie vektori ja kõigi sõnavara vektorite vahel, ning seejärel leiame minimaalse sõna indeksi, kasutades `argmin`.


Kuigi Word2Vec tundub suurepärane viis sõna semantika väljendamiseks, on sel mitmeid puudusi, sealhulgas järgmised:

* Nii CBoW kui ka skip-gram mudelid on **ennustavad sisestused**, mis arvestavad ainult lokaalset konteksti. Word2Vec ei kasuta ära globaalset konteksti.
* Word2Vec ei arvesta sõna **morfoloogiat**, st seda, et sõna tähendus võib sõltuda erinevatest sõna osadest, nagu näiteks tüvest.

**FastText** püüab ületada teist piirangut ja täiendab Word2Vec-i, õppides vektoriesitusi iga sõna ja sõnas leiduvate tähemärkide n-grammide jaoks. Esituste väärtused keskmistatakse igal treeningusammul üheks vektoriks. Kuigi see lisab eeltreeningule palju täiendavat arvutustööd, võimaldab see sõna sisestustel kodeerida alam-sõna teavet.

Teine meetod, **GloVe**, kasutab sõna sisestuste jaoks teistsugust lähenemist, mis põhineb sõna-konteksti maatriksi faktorisatsioonil. Kõigepealt koostab see suure maatriksi, mis loendab sõnade esinemiste arvu erinevates kontekstides, ja seejärel püüab esitada seda maatriksit madalamates dimensioonides viisil, mis minimeerib rekonstruktsiooni kaotuse.

Gensim teek toetab neid sõna sisestusi ning saate nendega katsetada, muutes ülaltoodud mudeli laadimise koodi.


## Eelõpetatud sisendvektorite kasutamine Kerases

Me saame ülaltoodud näidet muuta nii, et täidame oma sisendkihis oleva maatriksi semantiliste sisendvektoritega, nagu Word2Vec. Eelõpetatud sisendvektori ja tekstikorpuse sõnavarad ei pruugi kokku langeda, seega peame valima ühe. Siin uurime kahte võimalikku varianti: tokeniseerija sõnavara kasutamine ja Word2Vec sisendvektorite sõnavara kasutamine.

### Tokeniseerija sõnavara kasutamine

Tokeniseerija sõnavara kasutamisel on osadel sõnavara sõnadel vastavad Word2Vec sisendvektorid, kuid osad võivad puududa. Arvestades, et meie sõnavara suurus on `vocab_size` ja Word2Vec sisendvektori pikkus on `embed_size`, esitatakse sisendkiht kaalu maatriksina kujuga `vocab_size`$\times$`embed_size`. Täidame selle maatriksi, läbides sõnavara:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Sõnade jaoks, mis ei ole Word2Vec sõnavaras, võime need kas jätta nullideks või genereerida juhusliku vektori.

Nüüd saame määratleda sisestuskihi eeltreenitud kaaludega:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

Nüüd treenime oma mudelit.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: Pange tähele, et määrame `trainable=False`, kui loome `Embedding`, mis tähendab, et me ei treeni Embedding-kihi uuesti. See võib põhjustada veidi madalama täpsuse, kuid kiirendab treenimisprotsessi.

### Embedding-sõnavara kasutamine

Eelneva lähenemise üks probleem on see, et TextVectorization ja Embedding kasutavad erinevaid sõnavarasid. Selle probleemi lahendamiseks saame kasutada ühte järgmistest lahendustest:
* Treenida Word2Vec mudel uuesti meie sõnavara põhjal.
* Laadida meie andmestik, kasutades eelnevalt treenitud Word2Vec mudeli sõnavara. Sõnavara, mida kasutatakse andmestiku laadimiseks, saab määrata laadimise ajal.

Viimane lähenemine tundub lihtsam, seega rakendame seda. Kõigepealt loome `TextVectorization` kihi määratud sõnavaraga, mis on võetud Word2Vec embedding'utest:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensimi sõnaembeddingsi teek sisaldab mugavat funktsiooni `get_keras_embeddings`, mis loob automaatselt vastava Keras embeddingsi kihi teie jaoks.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Üks põhjusi, miks me ei näe suuremat täpsust, on see, et mõned sõnad meie andmestikust puuduvad eeltreenitud GloVe sõnavarast ja seetõttu neid sisuliselt eiratakse. Selle ületamiseks saame treenida omaenda sõnaembeddingsid, mis põhinevad meie andmestikul.


## Kontekstuaalsed sisendvektorid

Üks traditsiooniliste eeltreenitud sisendvektorite, nagu Word2Vec, peamisi piiranguid on see, et kuigi need suudavad tabada sõna mingit tähendust, ei suuda need eristada erinevaid tähendusi. See võib põhjustada probleeme järgnevatel mudelitel.

Näiteks sõnal 'play' (mängima/etendus) on erinev tähendus nendes kahes lauses:
- Ma käisin teatris **etendust** vaatamas.
- John tahab **mängida** oma sõpradega.

Eeltreenitud sisendvektorid, millest me rääkisime, esindavad mõlemat tähendust sõnast 'play' sama sisendvektoriga. Selle piirangu ületamiseks peame looma sisendvektorid, mis põhinevad **keelemudelil**, mis on treenitud suurel tekstikorpusel ja *teab*, kuidas sõnu saab erinevates kontekstides kokku panna. Kontekstuaalsete sisendvektorite arutelu jääb selle õpetuse ulatusest välja, kuid me tuleme nende juurde tagasi, kui räägime keelemudelitest järgmises osas.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
